In [6]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import modules.missing_values_module as mvm
import modules.feature_filters as filters


ModuleNotFoundError: No module named 'modules'

Тестовый датасет:

In [4]:
test_df = mvm.create_test_df(nans = False)
test_df

,x1,x2,x3,x4,xcat,target
0,0.404887,0.190837,0.107463,0.187022,d,0
1,-1.196204,-0.495785,0.327509,-0.003612,c,1
2,0.144787,0.049976,-0.273582,-1.98123,e,1
3,0.743311,0.387728,-0.989044,-0.756097,c,1
4,0.079919,0.012063,-0.005123,-0.424634,e,0
...,...,...,...,...,...,...
9995,-0.523808,0.442749,-0.360492,-0.383933,a,1
9996,-0.971413,-1.126833,-1.458249,0.933696,b,1
9997,0.263322,0.574945,1.786093,0.442066,d,1
9998,-0.728733,0.256484,0.922121,-0.696945,c,1


Смотрим, как работает фильтр

In [5]:
obj = filters.MutualCorrelationFilter(**{"n_features" : 1, "categorical_variables" : ["xcat", "target"]})
obj.fit(test_df)
obj.transform(test_df)

,x4,target,xcat
0,0.187022,0,d
1,-0.003612,1,c
2,-1.98123,1,e
3,-0.756097,1,c
4,-0.424634,0,e
...,...,...,...
9995,-0.383933,1,a
9996,0.933696,1,b
9997,0.442066,1,d
9998,-0.696945,1,c


Проверим, что улучшенный класс работает без ограничения набора столбцов:

In [8]:
#эта строка создает новый класс!
test_obj_class = teach_to_separate(PCA)
#эта создает инстанс класса!
test_obj = test_obj_class(categorical_variables = ["xcat"],
                         n_components = 2,
                         functionality = "PCA")
test_obj.fit(test_df.drop("target", axis = 1), test_df.target)
test_obj.transform(test_df.drop("target", axis = 1))

,PC1,PC2,xcat
0,0.186260,-0.436781,d
1,-0.014952,1.074415,c
2,-0.272339,0.786610,e
3,-0.664373,-0.330846,c
4,-0.024028,0.130706,e
...,...,...,...
9995,-0.110167,0.637421,a
9996,-1.838490,0.388100,b
9997,1.821717,-0.380172,d
9998,0.874094,1.000467,c


А если мы затребуем слишком много компонент, то:

In [9]:
#эта строка создает новый класс!
test_obj_class = teach_to_separate(PCA)
#эта создает инстанс класса!
test_obj = test_obj_class(categorical_variables = ["xcat"],
                         n_components = 10,
                         functionality = "PCA")
test_obj.fit(test_df.drop("target", axis = 1), test_df.target)
test_obj.transform(test_df.drop("target", axis = 1))

Number of components exceeds number of factors! Limiting it!


,PC1,PC2,PC3,PC4,xcat
0,0.186260,-0.436781,0.023911,0.105022,d
1,-0.014952,1.074415,-0.547287,-0.579986,c
2,-0.272339,0.786610,1.817031,0.213308,e
3,-0.664373,-0.330846,0.997687,0.854729,c
4,-0.024028,0.130706,0.412705,0.021719,e
...,...,...,...,...,...
9995,-0.110167,0.637421,0.091351,0.578920,a
9996,-1.838490,0.388100,-1.305378,-0.198923,b
9997,1.821717,-0.380172,-0.237897,-0.446298,d
9998,0.874094,1.000467,0.295388,-0.244219,c


Теперь определим, что нам нужно взять 2 наиболее скоррелированных с остальными столбца и превратить их в 1 компонент:

In [90]:
test_obj_class = teach_to_separate(PCA)
test_obj = test_obj_class(categorical_variables = ["xcat"],
                         n_components = 1,
                         functionality = "PCA",
                         subset_correlated = 2)
test_obj.fit(test_df.drop("target", axis = 1), test_df.target)
test_obj.transform(test_df.drop("target", axis = 1))

,PC1,xcat,x1,x2
0,0.426281,b,-0.148979,0.047278
1,-0.862910,c,-0.224155,0.397040
2,3.124958,d,-2.062259,-3.124808
3,-0.595299,d,0.214689,-0.011242
4,3.020202,d,-1.503699,-2.461427
...,...,...,...,...
9995,0.000422,d,1.155392,-0.139790
9996,-0.029515,d,0.424790,0.562618
9997,0.864111,a,-0.500323,-0.567591
9998,-2.156228,a,1.129244,2.491267


Теперь определим, что нам нужно взять половину столбцов так, чтобы в нее попали наиболее скоррелированные, и превратить их в 1 компонент:

In [10]:
test_obj_class = teach_to_separate(PCA)
test_obj = test_obj_class(categorical_variables = ["xcat"],
                         n_components = 1,
                         functionality = "PCA",
                         subset_correlated = 0.5,
                         type_subset = 'share')
test_obj.fit(test_df.drop("target", axis = 1), test_df.target)
test_obj.transform(test_df.drop("target", axis = 1))

,PC1,x3,xcat,x2
0,0.439410,0.107463,d,0.190837
1,-1.053510,0.327509,c,-0.495785
2,-0.838823,-0.273582,e,0.049976
3,0.278418,-0.989044,c,0.387728
4,-0.141255,-0.005123,e,0.012063
...,...,...,...,...
9995,-0.649637,-0.360492,a,0.442749
9996,-0.402665,-1.458249,b,-1.126833
9997,0.439169,1.786093,d,0.574945
9998,-0.980576,0.922121,c,0.256484


Теперь выделим переменные по их значимости для PCA. Алгоритм следующий:
1. На всех количественных переменных (не включенных в categorical_variables и принимающих более 2 уникальных значений) строится обычный PCA с количеством компонент, равным количеству переменных
2. Оставляется столько компонент, чтобы их cumulative explained variance ratio был максимален, но при этом меньше, чем значение, переданное в cum_var_desired
3. Для каждой из изначальных переменных считается сумма весов для всех этих компонент
4. Выбирается subset_pca переменных, для которых эта метрика максимальна

In [143]:
test_obj_class = teach_to_separate(PCA)
test_obj = test_obj_class(categorical_variables = ["xcat"],
                         n_components = 1,
                         functionality = "PCA",
                         subset_pca = 2,
                         cum_var_desired = 1)
test_obj.fit(test_df.drop("target", axis = 1), test_df.target)
test_obj.transform(test_df.drop("target", axis = 1))

,PC1,xcat,x3,x2
0,-0.991512,b,-0.408091,0.047278
1,-1.039934,c,0.881211,0.397040
2,-1.669313,d,-3.109953,-3.124808
3,-0.238708,d,0.608519,-0.011242
4,-0.236375,d,-3.015351,-2.461427
...,...,...,...,...
9995,1.294752,d,0.003556,-0.139790
9996,1.107598,d,0.031666,0.562618
9997,1.798468,a,-0.872613,-0.567591
9998,1.300468,a,2.160092,2.491267
